In [ ]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.linear_model
import sklearn.metrics

# Was ist Data Science?

Kern: Aus histoischen Daten Vorhersagen treffen.

Dafür gibt es viele Algorithmen.
Eine einfache und weit verbreitete ist die lineare Regression.

# Lineare Regression


Bei vielen Analysen wollen wir eine Zahl $y$, gegeben einige Eingabeparameter $x$, vorhersagen -- diese Problemklasse nennt man "Regression".
Ein Beispiel wäre, dass wir den Preis eines Hauses ($y$) prognostizieren wollen, gegeben einige Parameter ($x$) des Hauses (Anzahl Zimmer, Größe in $m²$, Alter, ...).

Wir wollen also die Ausgaben einer unbekannte Funktion $f : \mathbb{R}^n \to \mathbb{R}$ schätzen.
Genauer, wir haben $N$ Beispiele $(x_i, f(x_i)) : i=0,..,N$ gegeben und wollen nun $f(x)$ für einige neue Punkte $x$ prognostizieren.

Beispiel:

Wir verwenden einen Mini-Datensatz der 1960 von Nancy Howell zum Studium des Volkes der ǃKung gesammelt wurde (https://en.wikipedia.org/wiki/%C7%83Kung_people) (und nur Personen <= 25 Jahre)

In [ ]:
df = pd.read_csv("../data/Howell1_young.csv")

In [ ]:
df.head()

In [ ]:
df.plot.scatter(x="age", y="height")

In diesem Beispiel besteht $x$ nur aus einer Ausprägung: "age", im Allgemeinen ist $x$ aber ein Vektor $(x_1, ..., x_n)$

Bei *linerarer* Regression, nehmen wir als Ansatz/Modell eine lineare Funktion für $f$ an. In unserem Beispiel also $y = a\cdot x + b$.
Die beiden Koeffizienten $a$ und $b$ sind noch unbestimmt und sind die freien Parameter unseres Modells. Wir bestimmen $a$ und $b$ so, dass unsere Daten "gut" zu dem Modell passen.
Weil wir (viel) mehr Datenpunkte als Parameter haben wird der Zusammenhang $y = a\cdot x + b$ niemals *exakt* erfüllt sein (und das wäre sogar nichtmal gut!), wir wählen stattdessen $a$ und $b$ so, dass die *mittlere quadratische Abweichung* von $f$ und den Daten minimiert wird.

In anderen Worten, wir wählen $a,b$ so, dass im Mittel über alle Datenpunkte $(x,y)$, $(a\cdot x + b - y)^2$ möglichst klein wird.
(die Abweichung vom Modell zur Realität, wobei wir Abweichungen quadratisch werten, d.h. größere Abweichungen werden stärker bestraft.)

Nochmal genauer formuliert: Gegeben n Paare von Datenpunkten $(x^{(1)}, y^{(1)}), ..., (x^{(n)}, y^{(n)})$
suchen wir $a,b$ so dass der Ausdruck

$\frac{1}{n} \sum_{i=1}^n (a\cdot x^{(i)} + b - y^{(i)})^2$

minimal wird.

Zuerst teilen wir die Daten in 2 Teile: Trainingsmenge und eine Testmenge. Die Idee ist, dass wir unser Modell nur auf den Trainingsdaten anpassen und dann auf den Testdaten evaluieren.
Die Testdaten sollten nicht in die Modellkonstruktion einfließen. Die Trainings- und Test-daten sollten nicht zusammenhängen. So etwas kann passieren, wenn die beiden Datenmengen nicht disjukt sind oder einige Test- und Trainingspunkte stark korreliert sind (z.B. bei Zeitreihen -- z.B. das Wetter heute und morgen ist oft ähnlich))

In [ ]:
N = df.shape[0]
n = int(0.7 * N)

rng = np.random.default_rng(42) # get a random-number generator with fixed seed (-> reproducibility!)

training_indices = list(rng.choice(range(0, N), n))
test_indices = list(set(range(0, N)).difference(set(training_indices))) # super inefficient but we don't care for such small numbers

In [ ]:
linear_reg = sklearn.linear_model.LinearRegression()

x_train = df[["age"]].iloc[training_indices]
y_train = df["height"].iloc[training_indices]

x_test = df[["age"]].iloc[test_indices]
y_test = df["height"].iloc[test_indices]


linear_reg.fit(x_train,y_train) # berechnet a und b

a = linear_reg.coef_[0]
b = linear_reg.intercept_
print(f"a = {a}, b = {b}")

In [ ]:
ax = pd.DataFrame(list(zip(x_train.iloc[:,0],y_train))).plot.scatter(x=0, y=1)
y_pred_train = linear_reg.predict(x_train)
y_pred_test = linear_reg.predict(x_test)

ax.axline((0,b), slope=a, color="darkred")

## Modell-Evaluation


### Plot der Residuen

Erinnerung: wir haben Datenpunkte $(x^{(i)}, y^{(i)})$
Sei $\hat{y^{(i)}}$ die Vorhersage unseres Regressionsmodells für den $i$-ten Datenpunkt.
Für jedes $i$ ist die Abweichung $r_i = \hat{y^{(i)}} - y^{(i)}$. Diese nennen wir *Residuuen* und wenn unser Regressionsmodell auf die Daten passt, sollten diese Residuen keine systematischen Abweichungen ("Bias") zeigen.
Wenn wir sie also plotten sollten sie zufällig um $0$ streuen.


In [ ]:
r_train = y_pred_train - y_train
r_test = y_pred_test - y_test

In [ ]:
r_train.plot(style='.')
r_test.plot(style='.')


Wir sehen keine systematische Verzerrung, also scheint unsere lineare Regression schon mal gut.

### $R^2$

Das simpleste Modell für die Zielvariable, wäre einfach den Mittelwert der Trainingsdaten zu nehmen.
Wir wollen die mittlere quadratische Abweichung unseres Modells gegen dieses "Mittelwert-Modell" vergleichen.
Im schlechtesten Fall liefern wir das selbe, das soll einen Wert von $0$ ergeben. Im besten Fall ist die Abweichung von den Daten $0$, dann soll unsere Bewertung $1$ sein.

$R^2 = 1 - \frac{\frac{1}{n} \sum_i (y^{(i)} - \hat{y^{(i)} } )}{\frac{1}{n} \sum_i (y^{(i)} - \bar{y})} = 1 - \frac{\sum_i (y^{(i)} - \hat{y^{(i)} })}{\sum_i (y^{(i)} - \bar{y})}$

(Bemerkung: Wir verwenden hier die R^2 Definition für Lineare Regression *mit* Intercept (d.h. b))

Die Varianz (=Streuung) der Daten ist definiert als die mittlere quadratische Abweichung der Daten von ihrem Mittelwert $\bar{y}$:

$ \frac{1}{n} \sum_i (y^{(i)} - \bar{y})^2 $

Eine andere Interpretation von $R^2$ ist der Anteil der vom Modell "erklärten" Varianz der Daten. Eine Rechnung zeigt, dass man $R^2$ schreiben kann als

$R^2 = \frac{\sum_i (\hat{y^{(i)} } - \bar{y})}{\sum_i (y^{(i)} - \bar{y})}$

Oben steht die Varianz der Vorhersage, unten die Varianz der Daten, $R^2$ sagt also wie viel von der Varianz in den Daten sich durch die Vorhersage produzieren (und daher "erklären") lässt.

In [ ]:
print(f"Train R2: {sklearn.metrics.r2_score(y_pred_test, y_test)}")
print(f"Test R2: {sklearn.metrics.r2_score(y_pred_test, y_test)}")

## Over/Underfitting

Von "Overfitting" spricht man wenn das Modell "zu gut" zu den Daten passt, d.h. es passt sehr gut auf die Trainingsdaten, aber generalisiert nicht auf neue Daten wie z.B. den Testdaten.
Der Fehler (z.B. mittlere quadratische Abweichung) auf den Trainingsdaten ist sehr klein, auf den Testdaten aber sehr groß.
Das gegenteilige Problem ist "Underfitting", d.h. das Modell ist nicht komplex genug, um sich den Daten anzupassen. Der Fehler (gemessen auf den Trainingsdaten) ist selbst nach Anpassung an die Trainingsdaten noch groß.

(weitere Erklärung https://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html )

# Projekt: Toronto Bike Sharing

Wir haben Daten von Bike Share Toronto (https://bikesharetoronto.com/)
Für jede Reise werden Daten (z.B. von wo nach wo, wie lange dauerte die Fahrt) gesammelt, die online verfügbar sind (https://open.toronto.ca/dataset/bike-share-toronto-ridership-data/).

Wir wollen 1 Woche im Voraus vorhersagen, wie viele Räder an einem gewissen Tag an einer Station ausgeliehen werden.

Nach dem Installieren des Packets (siehe Setup.md)
Das Herunterladen und Bereinigen der Daten ist am Einfachsten über das Skript aus dem "entry_points" Verzeichnis:

``` $ poetry install ```

``` (bike-share-py3.10) $ python entry_points/load_data.py ```

Als Warnung: Die tägliche Arbeit eines Data Scientist besteht zu 80% daraus, Daten zu bereinigen und vielleicht zu 20% daraus statistische Modelle zu bauen und zu trainieren.

# Daten

Bike Share Toronto, ~630 Stationen und ~7000 Räder.

Die Datenfelder sind
* Trip ID [int]
* Trip Duration [s, int]
* Trip Start Station ID [int]
* Trip Start time [timestamp]
* Trip Start Station Name [string]
* Trip End Station ID [int]
* Trip End Time [timestamp]
* Trip End Station Name [string]
* Bike ID [int]
* User type [string, eigentlich boolean 0/1 (member/casual)]


# Schritte 
1. Mache Dich mit den Daten vertraut (download, Einlesen)
2. Hauptaufgabe:
    a) Für jeden Tag und Station voraussagen, wie viele Fahrräder dort ausgeliehen werden.
    b) Extrahiere aus den Daten weitere Features, die die Vorhersage verbessern.
3. Mögliche Erweiterungen (erstmal ein Modell für Frage 2 bauen und evaluieren!):
 * Nutze weitere Datenquellen um die Vorhersage aus 2) zu verbessern (z.B. Wetter https://toronto.weatherstats.ca/download.html).
 * Was könnten andere Ziel-Variablen als "tägliche Anzahl Ausleihvorgänge" sein, die vielleicht für die Betreiber hilfreich(er) sind?
 * ...

Hinweise für Schritt 1 (Daten herunterladen, säubern und aggregieren)

1. Das Environment bike_share aktivieren.

2. Daten herunterladen und säubern, Annahme: wir sind im Projekt-Verzeichnis!

``` (bike-share-py3.10) $ python entry_points/load_data.py ```

3. Tägliche Ausleihzahlen berechnen

``` (bike-share-py3.10) $ python entry_points/compute_counts.py ```

# Links

## Nützliche Dokumentationen
https://pandas.pydata.org/docs/user_guide/indexing.html

https://pandas.pydata.org/docs/user_guide/merging.html

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html

https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html

## weitere Links
https://bikesharetoronto.com/faq/

https://www.kaggle.com/code/yclaudel/see-the-flow-of-bikes/notebook
(da wird auch beschrieben, wie man an die Stations-Daten https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information kommt)

https://tellingstorieswithdata.com/inputs/pdfs/paper_one-2022-hudson_yuen.pdf

https://toronto.weatherstats.ca/download.html (3 Jahre = 1095 Tage)

https://opendata.stackexchange.com/questions/7793/age-weight-and-height-dataset (Referenz auf https://github.com/rmcelreath/rethinking/blob/master/data/Howell1.csv und https://github.com/rmcelreath/rethinking/blob/master/data/Howell2.csv)
